In [ ]:
#Import the Libraries
import math
import numpy as np 
import pandas as pd
import pandas_datareader as pdr
import yfinance as yfin
yfin.pdr_override()

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use ('fivethirtyeight')

In [ ]:
#Starting and ending date of the dataset
start_date = "2012-06-01"
end_date = "2022-06-12"

In [ ]:
#Get the Stock Quotes

data = pdr.DataReader(name="AMZN", data_source='yahoo', start=start_date, end=end_date)

#Show the data 
print(data)

In [ ]:
#Get the number of rows and columns in the data set
print(data.shape)

In [ ]:
#Visualisation of the history of closing prices
plt.figure(figsize = (16,8))
plt.title('Close Price History')
plt.plot(data['Close'])
plt.xlabel ('Date', fontsize = 18)
plt.ylabel('Close Price USD ($)', fontsize =18)
plt.show()

In [ ]:
#Create a new data frame with only the 'Close Column'
data2 = data.filter(['Close'])

#Convert dataframe to numpy array 
dataset = data2.values

#Get the number of rows to train the model on 
training_data_len = math.ceil (len(dataset) * .8)
print (training_data_len)

In [ ]:
#Data Scaling
scaler = MinMaxScaler(feature_range = (0,1))
scaled_data = scaler.fit_transform(dataset)
print(scaled_data)

In [ ]:
#Create scaled training Data Set
train_data = scaled_data[0:training_data_len, :]

#Split the data into x_train and y_train
x_train = []
y_train = []

for i in range (60, len(train_data)):
  x_train.append(train_data[i-60:i, 0])
  y_train.append(train_data[i, 0])
  if i<= 60:
    print(x_train)
    print(y_train)
    print()

In [ ]:
#Convert x_train and y_train into Numpy Arrays
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
#Data Reshaping
x_train = np.reshape (x_train, (x_train.shape[0], x_train.shape[1], 1)) 
print(x_train.shape)

In [ ]:
#Building the LSTM Model

model = Sequential ()
model.add(LSTM(50, return_sequences=True , input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences= False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
#Model Compilation
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
#Training the Model
model.fit(x_train, y_train, batch_size = 1 , epochs = 1)

In [ ]:
#Create a testrun dataset
#Create a new array containing scaled values from index 1960 to 2560
test_data = scaled_data[training_data_len -60:, :]
#Create new testrun data sets
x_test = []
y_test = dataset[training_data_len:, :]
for i in range (60, len(test_data)):
  x_test.append(test_data[i-60:i,0])


In [ ]:
#Convert data into a numpy array
x_test = np.array (x_test)

In [ ]:
#Reshape the Data
x_test = np.reshape (x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
#Get the Model's predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
#Get the Root Mean Squared Error (RMSE)
rmse = np.sqrt(np.mean (predictions - y_test)**2)
print (rmse)

In [ ]:
#Data Plotting
train = data [:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

#Data visualisation 
plt.figure(figsize =  (16,8))
plt.title('Model')
plt.xlabel('Date', fontsize = 18)
plt.ylabel('Close Price USD ($)', fontsize = 18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc = 'lower right')
plt.show()

In [ ]:
#Show the valid and predicted prices
print (valid)

In [ ]:
#Getting the quote
amazon_quote = pdr.DataReader('AMZN', data_source = 'yahoo', start = '2016-01-01', end = '2022-06-12')

#Create a new Data Frame
new_df = amazon_quote.filter(['Close'])

#Get the last 60 days closing price values and convert the dataframe to an array
last_60_days = new_df[-60:].values

#Scale the data to be 1 and 0 
last_60_days_scaled = scaler.transform(last_60_days)

#Create an empty list
X_test = []

#Append the past 60 days
X_test.append(last_60_days_scaled)

#Convert the X_test data to a numpy array
X_test = np.array(X_test)

#Data Reshaping
X_test = np.reshape (X_test, (X_test.shape [0], X_test.shape[1], 1))

#Get the predicted scaled price
pred_price = model.predict (X_test)

#Undo scaling
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)


In [ ]:
#Getting the quote
amazon_quote2 = pdr.DataReader('AMZN', data_source = 'yahoo', start = '2022-06-10', end = '2022-06-10')
print (amazon_quote2)